###Creating data ingestion in broze table from the volume
####/Volumes/datamaster/bronze/files/sales.csv


Ways of creating data
1. User Interface(from Volume)
2. Pyspark (data Frame style)
3. SQL
#####https://docs.databricks.com/aws/en/sql/language-manual/functions/read_files

In [0]:
file_name="/Volumes/datamaster/bronze/files/sales.csv"
dataframe=spark.read.csv((file_name),header=True,inferSchema=True)
dataframe.show(dataframe.count(),truncate=False)

In [0]:
dataframe.printSchema()

In [0]:
# create table and load data into table
#dataframe.write.saveAsTable("datamaster.bronze.sales")
cleandf=dataframe.dropna()## drop any rows that contain NULL values
(
    cleandf.write
      .format("delta")
      .option("overwriteSchema", "true")       # ignore missing null columns and aggressive create columns
      .mode("overwrite")                   # replace existing data
      .saveAsTable("datamaster.bronze.sales")
)
print("Table 'datamaster.bronze.sales' created")

In [0]:
%sql
--creating table datamaster.bronze.sales_sql_data
CREATE OR REPLACE TABLE datamaster.bronze.sales_sql_data AS
(
--selects the data which will be ready to be inserted in the created table
SELECT *,current_timestamp as ingestion_date  FROM read_files(
  '/Volumes/datamaster/bronze/files/sales.csv',
  format=>'csv'
)
-- the below statement is used to filter out the rows with null values
WHERE 
    order_id IS NOT NULL
    OR customer_id IS NOT NULL
    OR transaction_id IS NOT NULL
    OR product_id IS NOT NULL
    OR quantity IS NOT NULL
    OR discount_amount IS NOT NULL
    OR total_amount IS NOT NULL
    OR order_date IS NOT NULL

)
  

In [0]:
import pandas as pd
import numpy as np
import matplotlib as plt
df=cleandf.toPandas()
df.head(10)

In [0]:
%python
#df.info()
#df.astype({'order_id':'int32','customer_id':'float'})
#df.info()
#df.replace(np.nan,0,inplace=False)
#df.iloc[:,1:4]

In [0]:
new_df=df.drop_duplicates().count()
